# **Deep Learning With Python  -  CHAPTER 2**

- This code creates a fully organized system for loading, processing, training, and evaluating a neural network model on the **MNIST** dataset.

- The `DataLoader` class loads and preprocesses image data, while the `NeuralNetwork` class defines a **deep neural network** with two layers. The `Trainer` class handles model training using the processed data, whereas the `Evaluator` class assesses model performance and makes predictions.

- To optimize data processing, the `BatchGenerator` class manages **batching**, and the `MatrixOperations` class performs matrix operations such as **addition, multiplication, and ReLU activation**.

- Additionally, the `GradientComputation` class uses `GradientTape` to compute gradients. This **modular and flexible** structure allows for easy expansion and adaptation for more complex models.

In [3]:
import math
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [4]:
class DataLoader:
    def __init__(self):
        self.train_images, self.train_labels, self.test_images, self.test_labels = self.load_data()

    def load_data(self):
        (train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
        return train_images, train_labels, test_images, test_labels

    def preprocess_data(self):
        self.train_images = self.train_images.reshape((60000, 28 * 28)).astype("float32") / 255
        self.test_images = self.test_images.reshape((10000, 28 * 28)).astype("float32") / 255

    def get_data(self):
        return (self.train_images, self.train_labels), (self.test_images, self.test_labels)

    def show_sample(self, index=0):
        plt.imshow(self.train_images[index].reshape(28, 28), cmap=plt.cm.binary)
        plt.show()
        print(f"Label: {self.train_labels[index]}")

In [5]:
class NeuralNetwork:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        model = keras.Sequential([
            layers.Dense(512, activation="relu", input_shape=(28*28,)),
            layers.Dense(10, activation="softmax")
        ])
        model.compile(optimizer="rmsprop",
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])
        return model

    def get_model(self):
        return self.model

In [6]:
class Trainer:
    def __init__(self, model, train_images, train_labels):
        self.model = model
        self.train_images = train_images
        self.train_labels = train_labels

    def train(self, epochs=5, batch_size=128):
        self.model.fit(self.train_images, self.train_labels, epochs=epochs, batch_size=batch_size)

In [7]:
class Evaluator:
    def __init__(self, model, test_images, test_labels):
        self.model = model
        self.test_images = test_images
        self.test_labels = test_labels

    def evaluate(self):
        test_loss, test_acc = self.model.evaluate(self.test_images, self.test_labels)
        print(f"Test Accuracy: {test_acc:.2f}")

    def predict(self, index=0):
        test_sample = self.test_images[index:index+1]
        predictions = self.model.predict(test_sample)
        predicted_label = np.argmax(predictions[0])
        confidence = predictions[0][predicted_label]
        print(f"Predicted Label: {predicted_label}, Confidence: {confidence:.2f}")
        return predicted_label

In [8]:
class MatrixOperations:
    @staticmethod
    def naive_relu(x):
        assert len(x.shape) == 2
        x = x.copy()
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                x[i, j] = max(x[i, j], 0)
        return x

    @staticmethod
    def naive_add(x, y):
        assert len(x.shape) == 2 and x.shape == y.shape
        x = x.copy()
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                x[i, j] += y[i, j]
        return x

    @staticmethod
    def naive_dot(x, y):
        assert len(x.shape) == 1 and len(y.shape) == 1
        assert x.shape[0] == y.shape[0]
        z = 0.
        for i in range(x.shape[0]):
            z += x[i] * y[i]
        return z

In [9]:
class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next_batch(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [10]:
class GradientComputation:
    @staticmethod
    def compute_gradient(x):
        x = tf.Variable(x)
        with tf.GradientTape() as tape:
            y = 2 * x + 3
        grad = tape.gradient(y, x)
        return grad.numpy()

    @staticmethod
    def compute_matrix_gradient(X, W, b):
        X = tf.Variable(X)
        W = tf.Variable(W)
        b = tf.Variable(b)
        with tf.GradientTape() as tape:
            y = tf.matmul(X, W) + b
        grad_W, grad_b = tape.gradient(y, [W, b])
        return grad_W.numpy(), grad_b.numpy()

In [11]:
data_loader = DataLoader()
data_loader.preprocess_data()
(train_images, train_labels), (test_images, test_labels) = data_loader.get_data()

In [12]:
neural_network = NeuralNetwork()
trainer = Trainer(neural_network.get_model(), train_images, train_labels)
trainer.train(epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8730 - loss: 0.4396
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9645 - loss: 0.1154
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9784 - loss: 0.0720
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9847 - loss: 0.0523
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9889 - loss: 0.0372


In [13]:
evaluator = Evaluator(neural_network.get_model(), test_images, test_labels)
evaluator.evaluate()
evaluator.predict(index=0)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9751 - loss: 0.0800
Test Accuracy: 0.98
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Label: 7, Confidence: 1.00


7

In [14]:
matrix_op = MatrixOperations()
x = np.random.random((20, 100))
y = np.random.random((20, 100))
print("Relu applied:", matrix_op.naive_relu(x)[:5])

Relu applied: [[0.68210867 0.26844333 0.02346306 0.89044372 0.48982586 0.39193264
  0.80974622 0.84823082 0.03399332 0.59364864 0.36517648 0.25243852
  0.98412398 0.09793063 0.56823322 0.02079737 0.31813958 0.07290921
  0.49425955 0.72827252 0.8979239  0.8027512  0.79725841 0.9020959
  0.21481667 0.85393007 0.43115246 0.27770735 0.11662055 0.00496846
  0.45311272 0.08493915 0.80411333 0.11856711 0.58654525 0.32689178
  0.30231045 0.22702521 0.56435989 0.91340063 0.25368822 0.59081105
  0.50027662 0.19948799 0.6696955  0.04269278 0.92594544 0.94451136
  0.81189181 0.76905306 0.17290508 0.65060295 0.0425853  0.00436092
  0.25493984 0.20051494 0.47260625 0.610804   0.77183345 0.13231488
  0.96410415 0.86537455 0.99227072 0.59900793 0.5997046  0.01192103
  0.22846114 0.59305235 0.92663665 0.45049505 0.8978445  0.60853246
  0.37912253 0.57271089 0.71688682 0.66898853 0.42815106 0.64871435
  0.55088668 0.90435243 0.85254626 0.55329338 0.67092899 0.20998684
  0.47818045 0.36032613 0.93704192 

In [15]:
grad_calc = GradientComputation()
grad_x = grad_calc.compute_gradient(0.)
print("Gradient of y = 2x + 3 w.r.t x:", grad_x)

Gradient of y = 2x + 3 w.r.t x: 2.0
